In [1]:
import cohere
import pandas as pd
from tqdm import tqdm
tqdm.pandas()

In [4]:
prompt = """
### Instruction
Generate 10 question answer pairs from the given text regarding movie ` White Chapel  `
` White Chapel ` should be mentioned in each question
Output should be in a JSON list like [{"question":"", "answer"}, {"question":"", "answer":""},...].
### Context
Description: A father (Jack) and his daughter (Riley) move to a small town after a losing Riley's mother. Shortly after Riley gets a part time job in the local pub some of town residents start to disappear. 
Release Year: 2019 
Runtime: 47 
Genre: Horror 
Rating: 6.3 
Votes: 23.0
"""

In [5]:
co = cohere.Client('nmkRdEBefO4hDlTTkNao7xpy6udZhyiIYZ3WgFLd') # This is your prod API key
def generate_questions_cohere(prompt):
  response = co.generate(
    model='command',
    prompt=prompt,
    max_tokens=500,
    temperature=0.7,
    k=0,
    stop_sequences=[],
    return_likelihoods='NONE')
  return response.generations[0].text

In [6]:
generate_questions_cohere(prompt)

' Here are 10 question-answer pairs about the movie *White Chapel*:\n\n```json\n[\n  {\n    "question": "What is the name of the movie?",\n    "answer": "White Chapel"\n  },\n  {\n    "question": "What year was it released?",\n    "answer": "2019"\n  },\n  {\n    "question": "What is the genre of the movie?",\n    "answer": "Horror"\n  },\n  {\n    "question": "What is the rating of the movie?",\n    "answer": "6.3"\n  },\n  {\n    "question": "What is the vote count of the movie?",\n    "answer": "23.0"\n  },\n  {\n    "question": "What is the runtime of the movie?",\n    "answer": "47"\n  },\n  {\n    "question": "What is the main plot of the movie?",\n    "answer": "A father (Jack) and his daughter (Riley) move to a small town after a losing Riley\'s mother. Shortly after Riley gets a part-time job in the local pub, some of the town\'s residents start to disappear."\n  },\n  {\n    "question": "Who are the main characters in the movie?",\n    "answer": "Jack and Riley"\n  },\n  {\n 

In [7]:
def generate_cohere_prompt(title, data):
    return f"""
### Instruction
You are provided with details regarding the movie `{title.strip()}`. By only referring to the data in ###Context perform the following:
1. Generate 10 question answer pairs
2. The movie name `{title.strip()}` should be mentioned in each question surrounded by double quotes
3. Output should be in a JSON list like [{{"question":"", "answer"}}, {{"question":"", "answer":""}},...].
###Context
{data}
""".strip()

In [8]:
sample = pd.read_csv("sample_complete_desc.csv")

In [9]:
sample["cohere_prompt"] = sample.apply(lambda row: generate_cohere_prompt(row["originalTitle"], row["data"]), axis=1)

In [12]:
sample_df = sample.sample(50).copy()

In [13]:
sample_df["cohere_op"] = sample_df["cohere_prompt"].progress_apply(generate_questions_cohere)


100%|██████████| 50/50 [08:40<00:00, 10.41s/it]


In [16]:
sample_df[["originalTitle", "cohere_op"]].to_excel("cohere_op.xlsx", index=False)

# Validate Output

In [62]:
import pandas as pd
import regex as re

In [63]:
df = pd.read_feather(r"datasets/movie_datasets/imdb/movie_sample_desc_0.feather")

In [64]:
df.head(20)

,tconst,originalTitle,data,cohere_prompt,cohere_op
0,tt10024036,Satyarthi,Description: Kailash Satyarthi has spent his l...,### Instruction\nYou are provided with details...,Here are 10 question-answer pairs about the m...
1,tt11230106,Jasmine Star,Description: A determined 16-year-old with alb...,### Instruction\nYou are provided with details...,Here are 10 question-answer pairs about the m...
2,tt15017854,The Making of SI Swimsuit 2018,Description: Go behind the scenes with the cre...,### Instruction\nYou are provided with details...,Here are 10 question-answer pairs about the m...
3,tt12787498,"Boeing, Boeing","Description: Set in the late 1960s, ""Boeing Bo...",### Instruction\nYou are provided with details...,Here are 10 question-answer pairs about the m...
4,tt29347941,I Santo,Description: The film narrates everyday life a...,### Instruction\nYou are provided with details...,Here are 10 question-answer pairs about the m...
5,tt2619512,The Big Take,Description: A reclusive movie star is fingere...,### Instruction\nYou are provided with details...,Here are 10 question-answer pairs about the m...
6,tt10151562,Portrait of My Love,Description: Loving couple Miguel and Monay la...,### Instruction\nYou are provided with details...,Here are 10 question-answer pairs about the m...
7,tt10300166,Cha Ma,Description: A mysterious incidents keep happe...,### Instruction\nYou are provided with details...,Here are 10 question-answer pairs about the m...
8,tt11069570,Unni Ikki,Description: A young Punjabi couple doesn't re...,### Instruction\nYou are provided with details...,Here are 10 question-answer pairs about the m...
9,tt11018792,Ivanov,Description: A successful top manager of an in...,### Instruction\nYou are provided with details...,Here are 10 question-answer pairs about the m...


In [65]:
df.shape

(500, 5)

In [66]:
# AttributeError: due to incomplete generations
# SyntaxError: Malformed json
# NameError: key does not have quotes in the json

In [67]:
df["qa_list"] = None
counter = 0
for i, row in df.iterrows():
    try:
        # print(i)
        json_string = re.search(r"```json([\w\W]*?)```", row["cohere_op"]).group(1)
        df.at[i, "qa_list"] = eval(json_string)
    except (AttributeError, SyntaxError, NameError):
        counter+=1

In [68]:
counter

29

In [54]:
df.iloc[60].cohere_op

' Here are 10 question-answer pairs about the movie *Run for Dream*:\n```json\n[\n    {"question": "What is the name of the movie?", "answer": "Run for Dream"},\n    {"question": "When was the movie released?", "answer": "2019"},\n    {"question": "What is the genre of the movie?", "answer": "Documentary"},\n    {"question": "What is the rating of the movie?", "answer": "7.1"},\n    {"question": "How many votes does the movie have?", "answer": "40.0"},\n    {"question": "Who are the main characters in the movie?", "answer": "Chen Yen-Po, Tommy Chen"},\n    {"question": "What does the movie document?", "answer": "Chen Yen-Po\'s ten-year journey to become the champion of an ultra-marathon"},\n    {"question": "What is the runtime of the movie?", "answer": "102 minutes"},\n    {"question": "What is the native country of the main character?", "answer": "Taiwan"},\n    {"question": "Who is the director of the movie?", "answer": "Peter Ho-Sun"]\n]\n```'

## Extract information from cohere prompt 1

In [1]:
from functools import reduce
from glob import glob
import pandas as pd
import json
import regex as re
from tqdm import tqdm

In [2]:
dataset_paths = glob(r"datasets/movie_datasets/imdb/movie_sample_desc_?.feather")
print(dataset_paths)
print(len(dataset_paths))

['datasets/movie_datasets/imdb\\movie_sample_desc_0.feather', 'datasets/movie_datasets/imdb\\movie_sample_desc_1.feather', 'datasets/movie_datasets/imdb\\movie_sample_desc_2.feather', 'datasets/movie_datasets/imdb\\movie_sample_desc_3.feather', 'datasets/movie_datasets/imdb\\movie_sample_desc_4.feather', 'datasets/movie_datasets/imdb\\movie_sample_desc_5.feather', 'datasets/movie_datasets/imdb\\movie_sample_desc_6.feather']
7


In [3]:
datasets = map(pd.read_feather, dataset_paths)

In [4]:
df = reduce(lambda x,y: pd.concat([x, y], axis=0, ignore_index=True), datasets)

In [5]:
df.shape

(3500, 5)

In [6]:
df.head()

,tconst,originalTitle,data,cohere_prompt,cohere_op
0,tt10024036,Satyarthi,Description: Kailash Satyarthi has spent his l...,### Instruction\nYou are provided with details...,Here are 10 question-answer pairs about the m...
1,tt11230106,Jasmine Star,Description: A determined 16-year-old with alb...,### Instruction\nYou are provided with details...,Here are 10 question-answer pairs about the m...
2,tt15017854,The Making of SI Swimsuit 2018,Description: Go behind the scenes with the cre...,### Instruction\nYou are provided with details...,Here are 10 question-answer pairs about the m...
3,tt12787498,"Boeing, Boeing","Description: Set in the late 1960s, ""Boeing Bo...",### Instruction\nYou are provided with details...,Here are 10 question-answer pairs about the m...
4,tt29347941,I Santo,Description: The film narrates everyday life a...,### Instruction\nYou are provided with details...,Here are 10 question-answer pairs about the m...


In [7]:
df["qa_list"] = None
counter = 0
for i, row in tqdm(df.iterrows(), total=df.shape[0]):
    try:
        # print(i)
        json_string = re.search(r"```json([\w\W]*?)```", row["cohere_op"]).group(1)
        df.at[i, "qa_list"] = eval(json_string)
    except (AttributeError, SyntaxError, NameError):
        counter+=1

100%|████████████████████████████████████████████████████████████████████████████| 3500/3500 [00:00<00:00, 6168.97it/s]


In [8]:
counter

385

In [9]:
df = df.explode(["qa_list"], ignore_index=True)

In [10]:
df.iloc[131]

tconst                                                  tt13352604
originalTitle                                       Archiving Time
data             Description: Some movie superheroes save movie...
cohere_prompt    ### Instruction\nYou are provided with details...
cohere_op         Here are 10 question-answer pairs about the m...
qa_list          {'question': 'How many votes does the movie ha...
Name: 131, dtype: object

In [11]:
df["question"] = ""
df["answer"] = ""
for i, row in tqdm(df.iterrows(), total=df.shape[0]):
    # print(i)
    try:
        df.at[i, "question"] = row["qa_list"].get("question", None)
        df.at[i, "answer"] = row["qa_list"].get("answer", None)
    except AttributeError:
        df.at[i, "question"] = None
        df.at[i, "answer"] = None

100%|█████████████████████████████████████████████████████████████████████████| 32663/32663 [00:01<00:00, 18303.52it/s]


In [12]:
df.answer = df.answer.apply(lambda x: ", ".join(x) if type(x)==list else x)

In [13]:
df = df[~(df.question.isna()| df.answer.isna() | (df.question.str.strip()=="") | (df.answer.str.strip()==""))].reset_index(drop=True)

In [22]:
df.answer.apply(type).unique()

array([<class 'str'>, <class 'float'>, <class 'int'>], dtype=object)

In [24]:
df.answer = df.answer.astype(str)

In [25]:
df[["tconst", "originalTitle", "data", "question", "answer"]].to_feather("datasets/movie_datasets/imdb/movies_qa_cohere.feather")